In [1]:
from google.colab import files
uploaded = files.upload()

Saving embed_df.csv to embed_df.csv


In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [3]:
df = pd.read_csv('/content/embed_df.csv')


In [4]:
df['clean_category'] =  df['clean_category'].apply(lambda x:x.strip())
num_labels = len(set(df['clean_category']))

label2id, id2label = dict(), dict()
for i, label in enumerate(list(set(df['clean_category']))):
    label2id[label] = i
    id2label[str(i)] = label

texts = df['comment'].tolist()

labels = []
for ele in df['clean_category'].tolist():
  labels.append(label2id[ele])

In [5]:
class TextClassificationDataset(Dataset):

    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [6]:
label2id

{'Operational': 0,
 'Cost': 1,
 'Performance': 2,
 'Usablity': 3,
 'Others': 4,
 'Sales': 5}

In [7]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [8]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [9]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

In [18]:
def predict_sentiment(text, model, tokenizer, device, max_length=512):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    return id2label[str(preds.item())]

In [11]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 18
max_length = 512
batch_size = 4
num_epochs = 3
learning_rate = 1e-5

In [12]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [13]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [15]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train(model, train_dataloader, optimizer, scheduler, device)
    accuracy, report = evaluate(model, val_dataloader, device)
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(report)

Epoch 1/3
Validation Accuracy: 0.8095
              precision    recall  f1-score   support

           0       0.88      0.76      0.82       420
           1       0.89      0.85      0.87       102
           2       0.49      0.51      0.50        37
           3       0.81      0.91      0.86       416
           4       0.66      0.68      0.67       131
           5       0.80      0.80      0.80        91

    accuracy                           0.81      1197
   macro avg       0.75      0.75      0.75      1197
weighted avg       0.81      0.81      0.81      1197

Epoch 2/3
Validation Accuracy: 0.8112
              precision    recall  f1-score   support

           0       0.84      0.79      0.81       420
           1       0.89      0.81      0.85       102
           2       0.69      0.65      0.67        37
           3       0.84      0.88      0.86       416
           4       0.62      0.67      0.65       131
           5       0.80      0.86      0.83        91

 

In [19]:
df2 = pd.read_csv("/content/file2.csv")
df2['Q3 - Why'] = df2['Q3 - Why'].astype(str)

df2['Q3M1'] = df2.apply(lambda x: predict_sentiment(x['Q3 - Why'], model, tokenizer, device), axis=1)

df2['Q4 - What'] = df2['Q4 - What'].astype(str)

df2['Q4M1'] = df2.apply(lambda x: predict_sentiment(x['Q4 - What'], model, tokenizer, device), axis=1)

df2.to_csv('file3.csv')


In [20]:
files.download("file3.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
torch.save(model.state_dict(), "bert_classifier_6_class.pth")

In [ ]:
files.download("bert_classifier_6_class.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>